This notebook implements a generative function from the modlamp package to a library of 5,000 peptides using the amino acid composition of APD inliers.
Müller A. T. et al. (2017) modlAMP: Python for antimicrobial peptides, Bioinformatics 33, (17), 2753-2755, DOI:10.1093/bioinformatics/btx285.

#### Set working directory

In [1]:
import os
os.chdir('/Users/fabienplisson/Desktop/MODELS/')
print(os.getcwd())

/Users/fabienplisson/Desktop/MODELS


#### Modules

In [2]:
# Pip install modules
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install modlamp
!{sys.executable} -m pip install mysql.connector

     |████████████████████████████████| 1.5 MB 1.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 20.2
    Uninstalling pip-20.2:
      Successfully uninstalled pip-20.2


In [3]:
import pandas as pd
import numpy as np

#### Peptide Length APD Inliers and Outliers

In [4]:
io_apd = pd.read_csv('./Results/outliers_apd_hp1.csv', index_col=0)

# Create two lists (inliers_apd and outliers_apd) based on Average KNN labelling
outliers_apd_ix = io_apd[io_apd['Average KNN'] == 1].index
inliers_apd_ix = io_apd[io_apd['Average KNN'] == 0].index

In [5]:
# Extract sequence length from total_APD fasta
fasta_file =  './Data/total_APD.fasta'

with open(fasta_file, "r") as file_handle:
    contents = file_handle.read()
    for line in file_handle:
        if line.startswith(">"):
            print(line)

In [6]:
def parse_fasta(fname):
    identifiers = []
    sequences = []
    with open(fname, "r") as fh:
        for line in fh:
            if line.startswith(">"):
                line = line.replace('\n','')
                line = line.replace('>', '')
                identifiers.append(line)
            
    return identifiers

def parse_fasta2(fname):
    identifiers = []
    sequences = []
    with open(fname, "r") as fh:
        for line in fh:
            if line.startswith(">"):
                line = line.replace('\n','')
                identifiers.append(line)
            else:
                line = line.replace('\n','')
                sequences.append(line)
            
    return sequences

In [7]:
ids = parse_fasta(fasta_file)
seqs = parse_fasta2(fasta_file)

total_apd = pd.DataFrame(seqs, columns=['Sequence'], index=ids)
total_apd

,Sequence
AP00001,GLWSKIKEVGKEAAKAAAKAAGKAALGAVSEAV
AP00002,YVPLPNVPQPGRRPFPTFPGQGPFNPKIKWPQGY
AP00003,DGVKLCDVPSGTWSGHCGSSSKCSQQCKDREHFAYGGACHYQFPSV...
AP00004,NLCERASLTWTGNCGNTGHCDTQCRNWESAKHGACHKRGNWKCFCYFDC
AP00005,VFIDILDKVENAIHNAAQVGIGFAKPFEKLINPK
AP00006,GNNRPVYIPQPRPPHPRI
AP00007,GNNRPVYIPQPRPPHPRL
AP00008,RLCRIVVIRVCR
AP00009,RFRPPIRRPPIRPPFYPPFRPPIRPPIFPPIRPPFRPPLGPFP
AP00010,RRIRPRPPRLPRPRPRPLPFPRPGPRPIPRPLPFPRPGPRPIPRPL...


In [8]:
#Subset dataframe into inliers and outliers by indices
inliers_df = total_apd.loc[inliers_apd_ix]
outliers_df = total_apd.loc[outliers_apd_ix]

In [9]:
#Measure maximal and minimal length among sequences
print('APD Inliers:') 
print(inliers_df.Sequence.str.len().describe())
print(' ')
print('APD Outliers:') 
print(outliers_df.Sequence.str.len().describe())

APD Inliers:
count    2807.000000
mean       31.865693
std        17.135537
min         7.000000
25%        21.000000
50%        29.000000
75%        38.000000
max       131.000000
Name: Sequence, dtype: float64
 
APD Outliers:
count    274.000000
mean      44.332117
std       45.403635
min        2.000000
25%       10.000000
50%       22.000000
75%       63.750000
max      183.000000
Name: Sequence, dtype: float64


The minimum and maximum sequence length are too extreme, we selected the 25% and 75% percentiles, instead.

#### Generated inliers

In [10]:
from modlamp.sequences import Random

In [11]:
generated_inliers = Random(5000, 21, 38)
generated_inliers.generate_sequences(proba=[0.08, 0.06, 0.02, 0.02, 0.05, 0.11, 0.02, 0.07, 0.11, 0.11, 0.01, 0.03, 0.04, 0.02, 0.05, 0.06, 0.04, 0.06, 0.02, 0.02])

In [12]:
generated_inliers.sequences

['EKFKTLVRGSIIELARLRKCVKAGVGVALN',
 'PAVIIIFVPGFYIKTKNAARAICILRGLLSPCIATI',
 'LNFKKRKLLCGLGSCGLGKAPGLQQILSKWTVRSAAAG',
 'WFESLGKVAHGLCTTIKVCIAGGQRQGEPAAH',
 'FLPVPCSVKKLGGLKKHRRSWIHKLVNL',
 'VAKKLSLIGVVILVVLPKAGYFRATLLYHWC',
 'GGRLFSKGKCLKGAGALKAGVKSRTLVCVNSEK',
 'TGLIQSAGAFCFGKVGGRCGGKKE',
 'RPETIRKEKLIHTLSLAIVLGSTKPISKWNK',
 'SACSCKQVRSVAPMPNWDEKGKCYWAKKFGLLFK',
 'PLLFLLLWLIGLTCLVKRAGFPWGTDPIC',
 'RECIVASKLLEIGKVWAVKSQACTNKVAAL',
 'AQLGKGGAAPHLALAKIPKLFRLDPNAKCAIWAM',
 'KLFKIGPCVKLGKFNIRKPLNHFCCKGFIIRLP',
 'RRGCKFKCELCGGAILPALFTCNIIGA',
 'EICTFAFLGLRSGLKGRSGCAGRMATFKVGHMTI',
 'TVKWRTSEKNLCKRICFVSCNALIGFAD',
 'KVAIWYCLLFPGGTSAGFGNV',
 'LRNGGHLSKKLCKLWRRVIGRFSWG',
 'IIIPASASLLIPNIRLLRFGISAQGTGRGTTCRQVRV',
 'GKFQALKAFHALCVRVGVTKAPLAGYIELG',
 'IKFQSGLKRSSAILVAVGLLDKDKGLLLPIKGG',
 'NIFLLFNNIALNGCKWKLIGGKKVRLKCRAG',
 'VDYLHCGKNLGPKPGRPKRKLL',
 'PNATVVVKPYNGAGLLKSCMHIILGFL',
 'IAGAFNVVSRNSKINCKTTYLKLD',
 'DFQGISKKKAKNCHIPAQKSFGLCTATIIPGKRNVS',
 'LASGRKTSFARLHGIIKIRHKIAQGSKLDCDLHLHHC',
 'IVS

In [13]:
generated_inliers.filter_duplicates()

In [14]:
generated_inliers.seqnum

5000

In [15]:
def Repeat(x): 
    _size = len(x) 
    repeated = [] 
    for i in range(_size): 
        k = i + 1
        for j in range(k, _size): 
            if x[i] == x[j] and x[i] not in repeated: 
                repeated.append(x[i]) 
    return repeated 

print(Repeat(generated_inliers.sequences))

[]


In [17]:
import difflib
def matches(list1, list2):
    while True:
        mbs = difflib.SequenceMatcher(None, list1, list2).get_matching_blocks()
        if len(mbs) == 1: break
        for i, j, n in mbs[::-1]:
            if n > 0: yield list1[i: i + n]
            del list1[i: i + n]
            del list2[j: j + n]
            
list(matches(generated_inliers.sequences, total_apd.Sequence))

[]

No duplicated sequences within newly generated peptides and with total APD dataset.

#### Convert both lists into fasta files with gi_1...5000 (generated inliers)

In [18]:
ids = ['gi_{}'.format(i) for i in range(1, 5001)] 
generated_inliers_df = pd.DataFrame(generated_inliers.sequences, columns=['Sequence'], index=ids)
generated_inliers_df

,Sequence
gi_1,EKFKTLVRGSIIELARLRKCVKAGVGVALN
gi_2,PAVIIIFVPGFYIKTKNAARAICILRGLLSPCIATI
gi_3,LNFKKRKLLCGLGSCGLGKAPGLQQILSKWTVRSAAAG
gi_4,WFESLGKVAHGLCTTIKVCIAGGQRQGEPAAH
gi_5,FLPVPCSVKKLGGLKKHRRSWIHKLVNL
gi_6,VAKKLSLIGVVILVVLPKAGYFRATLLYHWC
gi_7,GGRLFSKGKCLKGAGALKAGVKSRTLVCVNSEK
gi_8,TGLIQSAGAFCFGKVGGRCGGKKE
gi_9,RPETIRKEKLIHTLSLAIVLGSTKPISKWNK
gi_10,SACSCKQVRSVAPMPNWDEKGKCYWAKKFGLLFK


In [21]:
def fasta_converter(database):
    '''Use the indices of the database and the column 'Sequence' to create a fasta file'''
    ofile = open('./Data/generated_inliers.fasta', "w")

    for i in range(len(database.Sequence)):

        ofile.write(">" + database.index[i] + "\n" +database.Sequence[i] + "\n")
        
    ofile.close()
    
fasta_converter(generated_inliers_df)

#### Calculation of physicochemical properties (modlamp)

In [24]:
import joblib 
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor

In [25]:
pepdesc_gi = PeptideDescriptor('./Data/generated_inliers.fasta', 'eisenberg')
globdesc_gi = GlobalDescriptor('./Data/generated_inliers.fasta')

In [26]:
pepdesc_gi.load_scale('eisenberg')
pepdesc_gi.calculate_global()  # calculate global Eisenberg hydrophobicity
pepdesc_gi.calculate_moment(append=True)
pepdesc_gi.load_scale('gravy')  # load GRAVY scale
pepdesc_gi.calculate_global(append=True)  # calculate global GRAVY hydrophobicity
pepdesc_gi.calculate_moment(append=True)  # calculate GRAVY hydrophobic moment
pepdesc_gi.load_scale('z3')  # load old Z scale
pepdesc_gi.calculate_autocorr(1, append=True)  # calculate global Z scale (=window1 autocorrelation)
pepdesc_gi.load_scale('z5')  # load old Z scale
pepdesc_gi.calculate_autocorr(1, append=True)  # calculate global Z scale (=window1 autocorrelation)
pepdesc_gi.load_scale('AASI')
pepdesc_gi.calculate_global(append=True)  # calculate global AASI index
pepdesc_gi.calculate_moment(append=True)  # calculate AASI index moment
pepdesc_gi.load_scale('ABHPRK')
pepdesc_gi.calculate_global(append=True)  # calculate ABHPRK feature 
pepdesc_gi.load_scale('argos')
pepdesc_gi.calculate_global(append=True)  # calculate global argos index
pepdesc_gi.calculate_moment(append=True)  # calculate argos index moment
pepdesc_gi.load_scale('bulkiness')
pepdesc_gi.calculate_global(append=True)  # calculate global bulkiness index
pepdesc_gi.calculate_moment(append=True)  # calculate bulkiness index moment
pepdesc_gi.load_scale('charge_phys')
pepdesc_gi.calculate_global(append=True)  # calculate global charge_phys index
pepdesc_gi.load_scale('charge_acid')
pepdesc_gi.calculate_global(append=True)  # calculate global charge_acid index
pepdesc_gi.load_scale('Ez')
pepdesc_gi.calculate_global(append=True)  # calculate global energies of insertion of amino acid side chains into lipid bilayers index
pepdesc_gi.load_scale('flexibility')
pepdesc_gi.calculate_global(append=True)  # calculate global flexibility scale
pepdesc_gi.calculate_moment(append=True)  # calculate flexibility moment
pepdesc_gi.load_scale('grantham')
pepdesc_gi.calculate_global(append=True)  # calculate global amino acid side chain composition, polarity and molecular volume
pepdesc_gi.load_scale('hopp-woods')
pepdesc_gi.calculate_global(append=True)  # calculate global Hopp-Woods hydrophobicity scale
pepdesc_gi.calculate_moment(append=True)  # calculate Hopp-Woods hydrophobicity moment
pepdesc_gi.load_scale('ISAECI')
pepdesc_gi.calculate_global(append=True) # calculate global ISAECI (Isotropic Surface Area (ISA) and Electronic Charge Index (ECI) of amino acid side chains) index
pepdesc_gi.load_scale('janin')
pepdesc_gi.calculate_global(append=True)  # calculate global Janin hydrophobicity scale
pepdesc_gi.calculate_moment(append=True)  # calculate Janin hydrophobicity moment
pepdesc_gi.load_scale('kytedoolittle')
pepdesc_gi.calculate_global(append=True)  # calculate global Kyte & Doolittle hydrophobicity scale
pepdesc_gi.calculate_moment(append=True)  # calculate Kyte & Doolittle hydrophobicity moment
pepdesc_gi.load_scale('levitt_alpha')
pepdesc_gi.calculate_global(append=True)  # calculate global Levitt alpha-helix propensity scale
pepdesc_gi.calculate_moment(append=True)  # calculate Levitt alpha-helix propensity moment
pepdesc_gi.load_scale('MSS')
pepdesc_gi.calculate_global(append=True)  # calculate global MSS index, graph-theoretical index that reflects topological shape and size of amino acid side chains
pepdesc_gi.calculate_moment(append=True)  # calculate MSS moment
pepdesc_gi.load_scale('MSW')
pepdesc_gi.calculate_global(append=True)  # calculate global MSW scale, Amino acid scale based on a PCA of the molecular surface based WHIM descriptor (MS-WHIM), extended to natural amino acids
pepdesc_gi.load_scale('pepArc')
pepdesc_gi.calculate_global(append=True) # calculate global pepArc, modlabs pharmacophoric feature scale, dimensions are: hydrophobicity, polarity, positive charge, negative charge, proline.
pepdesc_gi.load_scale('pepcats')
pepdesc_gi.calculate_global(append=True) # calculate global pepcats, modlabs pharmacophoric feature based PEPCATS scale
pepdesc_gi.load_scale('polarity')
pepdesc_gi.calculate_global(append=True)  # calculate global AA polarity 
pepdesc_gi.calculate_moment(append=True)  # calculate AA polarity moment
pepdesc_gi.load_scale('PPCALI')
pepdesc_gi.calculate_global(append=True)  # calculate global modlabs inhouse scale derived from a PCA of 143 amino acid property scales
pepdesc_gi.load_scale('refractivity')
pepdesc_gi.calculate_global(append=True) # calculate global relative AA refractivity
pepdesc_gi.calculate_moment(append=True) # calculate relative AA refractivity moment
pepdesc_gi.load_scale('t_scale')
pepdesc_gi.calculate_global(append=True) # calculate global t scale, A PCA derived scale based on amino acid side chain properties calculated with 6 different probes of the GRID program
pepdesc_gi.load_scale('TM_tend')
pepdesc_gi.calculate_global(append=True) # calculate global Amino acid transmembrane propensity scale
pepdesc_gi.calculate_moment(append=True) # calculate Amino acid transmembrane propensity scale moment

col_names1 = 'ID,Sequence,H_Eisenberg,uH_Eisenberg,H_GRAVY,uH_GRAVY,Z3_1,Z3_2,Z3_3, Z5_1,Z5_2,Z5_3, Z5_4,Z5_5,S_AASI, uS_AASI, modlas_ABHPRK, H_argos, uH_argos, B_Builkiness, uB_Builkiness, charge_phys, charge_acid, Ez, flexibility, u_flexibility, Grantham, H_HoppWoods, uH-HoppWoods, ISAECI, H_Janin, uH_Janin, H_KyteDoolittle, uH_KyteDoolittle, F_Levitt, uF_Levitt, MSS_shape, u_MSS_shape, MSW, pepArc, pepcats, polarity, u_polarity, PPCALI, refractivity, u_refractivity, t_scale, TM_tend, u_TM_tend'
pepdesc_gi.save_descriptor('./Descriptors/pep_descriptors_gi.csv', header=col_names1)

globdesc_gi.length()  # sequence length
globdesc_gi.boman_index(append=True)  # Boman index
globdesc_gi.aromaticity(append=True)  # global aromaticity
globdesc_gi.aliphatic_index(append=True)  # aliphatic index
globdesc_gi.instability_index(append=True)  # instability index
globdesc_gi.calculate_charge(ph=7.4, amide=False, append=True)  # net charge
globdesc_gi.calculate_MW(amide=False, append=True)  # molecular weight
globdesc_gi.isoelectric_point(amide=False,append=True) # isoelectric point
globdesc_gi.hydrophobic_ratio(append=True)

col_names2 = 'ID, Sequence,Length,BomanIndex,Aromaticity,AliphaticIndex,InstabilityIndex, NetCharge, MW, IsoelectricPoint, HydrophobicRatio'
globdesc_gi.save_descriptor('./Descriptors/global_descriptors_gi.csv', header=col_names2)

In [30]:
pepdesc_gi = pd.read_csv('./Descriptors/pep_descriptors_gi.csv', index_col=0)
globdesc_gi = pd.read_csv('./Descriptors/global_descriptors_gi.csv', index_col=0)
generated_inliers_props = pepdesc_gi.join(globdesc_gi)

In [34]:
generated_inliers_props.head(10)

,Sequence,H_Eisenberg,uH_Eisenberg,H_GRAVY,uH_GRAVY,Z3_1,Z3_2,Z3_3,Z5_1,Z5_2,...,Sequence,Length,BomanIndex,Aromaticity,AliphaticIndex,InstabilityIndex,NetCharge,MW,IsoelectricPoint,HydrophobicRatio
# ID,,,,,,,,,,,,,,,,,,,,,
b'gi_1',b'EKFKTLVRGSIIELARLRKCVKAGVGVALN',b'0.013333333333333367',b'0.35016685453050556',b'0.3133333333333334',b'1.0718660497145227',b'8.98049',b'5.657523333333334',b'3.6037666666666675',b'8.491113333333335',b'4.778333333333333',...,b'EKFKTLVRGSIIELARLRKCVKAGVGVALN',b'30.0',b'1.1666666666666665',b'0.03333333333333333',b'126.66666666666666',b'16.59',b'4.835',b'3269.95',b'11.03125',b'0.5'
b'gi_2',b'PAVIIIFVPGFYIKTKNAARAICILRGLLSPCIATI',b'0.425',b'0.20307690983656168',b'1.2777777777777777',b'0.726887076913708',b'9.423908333333333',b'4.281316666666667',b'3.1303222222222216',b'8.368783333333333',b'4.151866666666667',...,b'PAVIIIFVPGFYIKTKNAARAICILRGLLSPCIATI',b'36.0',b'-0.7052777777777774',b'0.08333333333333333',b'149.16666666666666',b'45.57250000000001',b'3.679',b'3857.77',b'10.2333984375',b'0.6111111111111112'
b'gi_3',b'LNFKKRKLLCGLGSCGLGKAPGLQQILSKWTVRSAAAG',b'0.07105263157894737',b'0.12303326261302082',b'0.08684210526315779',b'0.3403406082901986',b'8.371739473684215',b'6.745050000000001',b'3.4009631578947377',b'7.9112315789473655',b'5.129831578947368',...,b'LNFKKRKLLCGLGSCGLGKAPGLQQILSKWTVRSAAAG',b'38.0',b'0.6297368421052632',b'0.05263157894736842',b'100.26315789473682',b'45.90789473684211',b'6.679',b'3971.76',b'11.17333984375',b'0.42105263157894735'
b'gi_4',b'WFESLGKVAHGLCTTIKVCIAGGQRQGEPAAH',b'0.1571875',b'0.3934036808788779',b'0.01874999999999996',b'1.2178671727584867',b'7.28796875',b'6.953050000000001',b'2.7729937500000004',b'6.560362499999999',b'5.591081249999999',...,b'WFESLGKVAHGLCTTIKVCIAGGQRQGEPAAH',b'32.0',b'0.6565625',b'0.0625',b'79.375',b'26.290625000000002',b'0.765',b'3365.85',b'8.1064453125',b'0.40625'
b'gi_5',b'FLPVPCSVKKLGGLKKHRRSWIHKLVNL',b'-0.026428571428571395',b'0.19846807632620003',b'-0.125',b'0.8899384192573642',b'9.856521428571428',b'4.942992857142859',b'4.13384642857143',b'9.364928571428573',b'4.161446428571428',...,b'FLPVPCSVKKLGGLKKHRRSWIHKLVNL',b'28.0',b'1.0799999999999998',b'0.07142857142857142',b'114.64285714285715',b'70.88214285714285',b'6.917',b'3254.99',b'11.626953125',b'0.39285714285714285'
b'gi_6',b'VAKKLSLIGVVILVVLPKAGYFRATLLYHWC',b'0.42032258064516126',b'0.16849849583593676',b'1.1838709677419355',b'0.6269488265660162',b'9.242506451612904',b'5.171370967741935',b'2.7191419354838713',b'8.902370967741936',b'4.918403225806451',...,b'VAKKLSLIGVVILVVLPKAGYFRATLLYHWC',b'31.0',b'-0.9993548387096773',b'0.12903225806451613',b'157.09677419354838',b'19.677419354838708',b'3.872',b'3472.29',b'10.291259765625',b'0.5806451612903226'
b'gi_7',b'GGRLFSKGKCLKGAGALKAGVKSRTLVCVNSEK',b'-0.05727272727272725',b'0.11404785829683234',b'-0.11818181818181818',b'0.03491445221714836',b'7.400236363636362',b'7.502945454545456',b'3.944984848484849',b'6.9977',b'5.460324242424245',...,b'GGRLFSKGKCLKGAGALKAGVKSRTLVCVNSEK',b'33.0',b'1.2409090909090907',b'0.030303030303030304',b'82.72727272727273',b'9.778787878787883',b'6.679',b'3364.01',b'10.939208984375',b'0.3939393939393939'
b'gi_8',b'TGLIQSAGAFCFGKVGGRCGGKKE',b'0.09833333333333331',b'0.09033451416539139',b'-0.03333333333333329',b'0.3509660323210613',b'7.506870833333334',b'10.100487500000002',b'3.493841666666667',b'6.410466666666667',b'6.906195833333332',...,b'TGLIQSAGAFCFGKVGGRCGGKKE',b'24.0',b'0.7204166666666666',b'0.08333333333333333',b'52.91666666666666',b'16.949999999999996',b'2.681',b'2371.75',b'9.76611328125',b'0.375'
b'gi_9',b'RPETIRKEKLIHTLSLAIVLGSTKPISKWNK',b'-0.08387096774193548',b'0.20314050600508562',b'-0.38709677419354843',b'0.41231827977303154',b'9.450390322580647',b'3.7960806451612905',b'3.299454838709677',b'8.834312903225806',b'3.353261290322581',...,b'RPETIRKEKLIHTLSLAIVLGSTKPISKWNK',b'31.0',b'1.675483870967741',b'0.03225806451612903',b'113.2258064516129',b'26.75483870967742',b'5.03',b'3558.23',b'11.1728515625',b'0.3225806451612903'


#### Cleaning up byte issues with new datasets

In [35]:
# Removing duplicated columns
generated_inliers_props = generated_inliers_props.drop(['Sequence',' Sequence'], axis=1)

# Cleaning up 'b' character
cleaned_generated_inliers_props = pd.DataFrame()
for col in generated_inliers_props.columns:
    s = pd.Series(list(generated_inliers_props[col]))
    s = s.str.extract('[-+]?(\d+([.,]\d+))')[0]
    cleaned_generated_inliers_props[col]=s.values

cleaned_generated_inliers_props.index = ids
cleaned_generated_inliers_props = cleaned_generated_inliers_props.astype(float)
cleaned_generated_inliers_props

,H_Eisenberg,uH_Eisenberg,H_GRAVY,uH_GRAVY,Z3_1,Z3_2,Z3_3,Z5_1,Z5_2,Z5_3,...,u_TM_tend,Length,BomanIndex,Aromaticity,AliphaticIndex,InstabilityIndex,NetCharge,MW,IsoelectricPoint,HydrophobicRatio
gi_1,0.013333,0.350167,0.313333,1.071866,8.980490,5.657523,3.603767,8.491113,4.778333,3.526820,...,0.636639,30.0,1.166667,0.033333,126.666667,16.590000,4.835,3269.95,11.031250,0.500000
gi_2,0.425000,0.203077,1.277778,0.726887,9.423908,4.281317,3.130322,8.368783,4.151867,3.299603,...,0.414801,36.0,0.705278,0.083333,149.166667,45.572500,3.679,3857.77,10.233398,0.611111
gi_3,0.071053,0.123033,0.086842,0.340341,8.371739,6.745050,3.400963,7.911232,5.129832,3.194492,...,0.283085,38.0,0.629737,0.052632,100.263158,45.907895,6.679,3971.76,11.173340,0.421053
gi_4,0.157188,0.393404,0.018750,1.217867,7.287969,6.953050,2.772994,6.560362,5.591081,2.571663,...,0.734017,32.0,0.656563,0.062500,79.375000,26.290625,0.765,3365.85,8.106445,0.406250
gi_5,0.026429,0.198468,0.125000,0.889938,9.856521,4.942993,4.133846,9.364929,4.161446,3.670282,...,0.537696,28.0,1.080000,0.071429,114.642857,70.882143,6.917,3254.99,11.626953,0.392857
gi_6,0.420323,0.168498,1.183871,0.626949,9.242506,5.171371,2.719142,8.902371,4.918403,2.737161,...,0.374908,31.0,0.999355,0.129032,157.096774,19.677419,3.872,3472.29,10.291260,0.580645
gi_7,0.057273,0.114048,0.118182,0.034914,7.400236,7.502945,3.944985,6.997700,5.460324,3.470327,...,0.131483,33.0,1.240909,0.030303,82.727273,9.778788,6.679,3364.01,10.939209,0.393939
gi_8,0.098333,0.090335,0.033333,0.350966,7.506871,10.100488,3.493842,6.410467,6.906196,3.101304,...,0.034283,24.0,0.720417,0.083333,52.916667,16.950000,2.681,2371.75,9.766113,0.375000
gi_9,0.083871,0.203141,0.387097,0.412318,9.450390,3.796081,3.299455,8.834313,3.353261,3.063287,...,0.379212,31.0,1.675484,0.032258,113.225806,26.754839,5.030,3558.23,11.172852,0.322581
gi_10,0.060588,0.241601,0.441176,0.549779,7.993341,4.449276,4.356871,7.451109,4.051962,3.595459,...,0.421192,34.0,1.300882,0.147059,48.823529,46.311765,4.523,3907.62,9.906738,0.382353


In [36]:
#drops = cleaned_generated_inliers_props.columns[[0,48]]
#cleaned_generated_inliers_props = cleaned_generated_inliers_props.drop(columns=drops, axis=1)
#cleaned_generated_inliers_props

In [37]:
cleaned_generated_inliers_props.to_csv('./Descriptors/generated_inliers_props.csv')

#### Normalize both datasets to HemoPI-1 datasets

In [38]:
HemoPI1_model = pd.read_csv('./Data/HemoPI1_model.csv', index_col=0)
HemoPI1_validation = pd.read_csv('./Data/HemoPI1_validation.csv', index_col=0)

HemoPI1_model = HemoPI1_model.dropna(axis='columns')
HemoPI1_validation = HemoPI1_validation.dropna(axis='columns')

cleaned_HemoPI1_model = HemoPI1_model.drop(['Sequence',' Sequence', 'y_model_2cl'], axis=1)
cleaned_HemoPI1_validation = HemoPI1_validation.drop(['Sequence',' Sequence', 'y_validation_2cl'], axis=1)

print(cleaned_HemoPI1_model.shape, cleaned_HemoPI1_validation.shape)

(884, 56) (220, 56)


In [39]:
def normalize_with_model(df, df2):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df2[feature_name].max()
        min_value = df2[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [40]:
norm_generated_inliers = normalize_with_model(cleaned_generated_inliers_props, cleaned_HemoPI1_model)
norm_generated_inliers

,H_Eisenberg,uH_Eisenberg,H_GRAVY,uH_GRAVY,Z3_1,Z3_2,Z3_3,Z5_1,Z5_2,Z5_3,...,u_TM_tend,Length,BomanIndex,Aromaticity,AliphaticIndex,InstabilityIndex,NetCharge,MW,IsoelectricPoint,HydrophobicRatio
gi_1,0.528248,0.308351,0.567786,0.406958,0.379147,0.382558,0.344487,0.399769,0.471235,0.371109,...,0.365436,0.276596,0.467794,0.050000,0.432712,0.215197,0.368752,0.268346,0.816239,0.590909
gi_2,0.739149,0.178826,0.733370,0.275979,0.407108,0.261223,0.293469,0.389391,0.388393,0.344428,...,0.238099,0.340426,0.419787,0.125000,0.509576,0.320857,0.341839,0.323608,0.739640,0.722222
gi_3,0.557818,0.108341,0.528901,0.129218,0.340760,0.478440,0.322633,0.350576,0.517717,0.332085,...,0.162493,0.361702,0.411926,0.078947,0.342514,0.322079,0.411683,0.334324,0.829880,0.497608
gi_4,0.601946,0.346424,0.517210,0.462391,0.272418,0.496779,0.254963,0.235978,0.578712,0.258948,...,0.421332,0.297872,0.414718,0.093750,0.271157,0.250562,0.273997,0.277362,0.535440,0.480114
gi_5,0.534957,0.174767,0.535452,0.337885,0.434388,0.319560,0.401609,0.473897,0.389660,0.387956,...,0.308642,0.255319,0.458777,0.107143,0.391637,0.413126,0.417223,0.266940,0.873430,0.464286
gi_6,0.736753,0.148377,0.717247,0.238035,0.395669,0.339696,0.249160,0.434657,0.489758,0.278382,...,0.215200,0.287234,0.450385,0.193548,0.536666,0.226453,0.346332,0.287369,0.745195,0.686217
gi_7,0.550759,0.100429,0.534281,0.013256,0.279498,0.545261,0.381257,0.273079,0.561420,0.364475,...,0.075473,0.308511,0.475519,0.045455,0.282609,0.190366,0.411683,0.277189,0.807402,0.465565
gi_8,0.571795,0.079547,0.519714,0.133252,0.286222,0.774275,0.332642,0.223262,0.752620,0.321142,...,0.019679,0.212766,0.421362,0.125000,0.180771,0.216510,0.318604,0.183906,0.694778,0.443182
gi_9,0.564385,0.178882,0.580451,0.156546,0.408778,0.218442,0.311695,0.428883,0.282787,0.316678,...,0.217671,0.287234,0.520737,0.048387,0.386796,0.252254,0.373292,0.295448,0.829833,0.381232
gi_10,0.552457,0.212750,0.589736,0.208736,0.316898,0.276031,0.425642,0.311543,0.375182,0.379169,...,0.241768,0.319149,0.481760,0.220588,0.166788,0.323552,0.361488,0.328294,0.708279,0.451872


In [41]:
norm_generated_inliers.to_csv('./Descriptors/norm_generated_inliers.csv')

#### Predicting haemolytic activities for these datasets

In [42]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import scipy
import sklearn

In [43]:
from os import path

from sklearn import model_selection, metrics
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

In [44]:
from os import path
from sklearn import metrics
from sklearn.metrics import accuracy_score, cohen_kappa_score, roc_auc_score, matthews_corrcoef, average_precision_score
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC

In [46]:
seed=42
kfold = model_selection.KFold(n_splits=10, random_state=seed, shuffle= True)

In [47]:
X_generated_inliers = norm_generated_inliers

In [48]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [49]:
norm_HemoPI1_model = normalize(cleaned_HemoPI1_model)
norm_HemoPI1_validation = normalize(cleaned_HemoPI1_validation)

X_HemoPI1_model = norm_HemoPI1_model
X_HemoPI1_validation = norm_HemoPI1_validation 

y_HemoPI1_model = HemoPI1_model['y_model_2cl']
y_HemoPI1_validation = HemoPI1_validation['y_validation_2cl'] 

##### Model 1.1

In [50]:
from sklearn.feature_selection import RFECV

model1_hpi1 = LinearDiscriminantAnalysis(solver='svd', tol=0.0001)

rfecv_model = RFECV(model1_hpi1, step=1, cv=kfold)
rfecv = rfecv_model.fit(X_HemoPI1_model, y_HemoPI1_model)

X_HemoPI1_model_RFE = rfecv.transform(X_HemoPI1_model)
X_HemoPI1_val_RFE = rfecv.transform(X_HemoPI1_validation)
X_generated_inliers_RFE = rfecv.transform(X_generated_inliers)

print(X_generated_inliers_RFE.shape)

(5000, 16)


##### Model 1.2

In [52]:
model2_hpi1 = GradientBoostingClassifier(n_estimators=240, max_depth=4, min_samples_leaf=10, max_features='sqrt', random_state=seed)

corr_matrix = norm_HemoPI1_model.corr()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]

X_HemoPI1_model_trim = norm_HemoPI1_model.drop(norm_HemoPI1_model[to_drop], axis=1)
X_HemoPI1_val_trim = norm_HemoPI1_validation.drop(norm_HemoPI1_validation[to_drop], axis=1)

X_generated_inliers_trim = X_generated_inliers[list(X_HemoPI1_model_trim.columns)]

print(X_generated_inliers_trim.shape)

(5000, 26)


##### Model 1.3

In [53]:
model3_hpi1 = GradientBoostingClassifier(n_estimators=208, max_depth=4, min_samples_leaf=10, max_features='sqrt', random_state=seed)

X_generated_inliers = norm_generated_inliers

print(X_generated_inliers.shape)

(5000, 56)


##### Results for generated inliers

In [54]:
#Save test set results into dataframe
# Model 1.1
model1_hpi1.fit(X_HemoPI1_model_RFE, y_HemoPI1_model)
class_df = pd.DataFrame(model1_hpi1.predict(X_generated_inliers_RFE))
probs_df = pd.DataFrame(model1_hpi1.predict_proba(X_generated_inliers_RFE))

model1_hpi1_gi = class_df.merge(probs_df, how='outer', left_index=True, right_index=True)
model1_hpi1_gi.index = X_generated_inliers.index
model1_hpi1_gi.columns = ['model1_class_preds', 'model1_probability_0', 'model1_probability_1']

# Model 1.2
model2_hpi1.fit(X_HemoPI1_model_trim, y_HemoPI1_model)
class_df2 = pd.DataFrame(model2_hpi1.predict(X_generated_inliers_trim))
probs_df2 = pd.DataFrame(model2_hpi1.predict_proba(X_generated_inliers_trim))

model2_hpi1_gi = class_df2.merge(probs_df2, how='outer', left_index=True, right_index=True)
model2_hpi1_gi.index = X_generated_inliers.index
model2_hpi1_gi.columns = ['model2_class_preds', 'model2_probability_0', 'model2_probability_1']

# Model 1.3
model3_hpi1.fit(X_HemoPI1_model, y_HemoPI1_model)
class_df3 = pd.DataFrame(model3_hpi1.predict(X_generated_inliers))
probs_df3 = pd.DataFrame(model3_hpi1.predict_proba(X_generated_inliers))

model3_hpi1_gi = class_df3.merge(probs_df3, how='outer', left_index=True, right_index=True)
model3_hpi1_gi.index = X_generated_inliers.index
model3_hpi1_gi.columns = ['model3_class_preds', 'model3_probability_0', 'model3_probability_1']

# Merge model dataframes
models_hpi1 = [model1_hpi1_gi, model2_hpi1_gi, model3_hpi1_gi]

results_models_hpi1_gi = pd.concat(models_hpi1, axis=1, join='inner')
results_models_hpi1_gi.index = model1_hpi1_gi.index
results_models_hpi1_gi

#Create consensus columns
results_models_hpi1_gi['consensus_class_preds'] = results_models_hpi1_gi[['model1_class_preds', 'model2_class_preds', 'model3_class_preds']].mean(axis=1).astype(int)
results_models_hpi1_gi['consensus_probability_0'] = results_models_hpi1_gi[['model1_probability_0', 'model2_probability_0', 'model3_probability_0']].mean(axis=1)
results_models_hpi1_gi['consensus_probability_1'] = results_models_hpi1_gi[['model1_probability_1', 'model2_probability_1', 'model3_probability_1']].mean(axis=1)

results_models_hpi1_gi

,model1_class_preds,model1_probability_0,model1_probability_1,model2_class_preds,model2_probability_0,model2_probability_1,model3_class_preds,model3_probability_0,model3_probability_1,consensus_class_preds,consensus_probability_0,consensus_probability_1
gi_1,0,0.755229,0.244771,1,0.008633,0.991367,1,0.031063,0.968937,0,0.264975,0.735025
gi_2,0,0.708206,0.291794,1,0.018363,0.981637,1,0.003990,0.996010,0,0.243520,0.756480
gi_3,1,0.435451,0.564549,1,0.002704,0.997296,1,0.003621,0.996379,1,0.147258,0.852742
gi_4,0,0.980356,0.019644,0,0.944337,0.055663,0,0.969352,0.030648,0,0.964682,0.035318
gi_5,1,0.013438,0.986562,1,0.023670,0.976330,1,0.000816,0.999184,1,0.012641,0.987359
gi_6,0,0.897992,0.102008,1,0.095480,0.904520,1,0.369263,0.630737,0,0.454245,0.545755
gi_7,0,0.737073,0.262927,1,0.000759,0.999241,1,0.126651,0.873349,0,0.288161,0.711839
gi_8,0,0.997270,0.002730,0,0.737739,0.262261,0,0.945359,0.054641,0,0.893456,0.106544
gi_9,1,0.248163,0.751837,0,0.992315,0.007685,0,0.915766,0.084234,0,0.718748,0.281252
gi_10,1,0.001508,0.998492,1,0.087943,0.912057,1,0.010380,0.989620,1,0.033277,0.966723


In [55]:
results_models_hpi1_gi.to_csv('./Results/predictions_generated_inliers_HemoPI1.csv')

#### Selected Generated Inliers 

non-hemolytic peptides with probability <0.25 and OS <0.25

In [56]:
selected_gi = pd.read_csv('./Data/selected_inliers.csv', index_col=0, header=-1)
gi_idx = list(selected_gi.index)

In [57]:
def parse_fasta(fname):
    identifiers = []
    sequences = []
    with open(fname, "r") as fh:
        for line in fh:
            if line.startswith(">"):
                line = line.replace('\n','')
                line = line.replace('>', '')
                identifiers.append(line)
            
    return identifiers

def parse_fasta2(fname):
    identifiers = []
    sequences = []
    with open(fname, "r") as fh:
        for line in fh:
            if line.startswith(">"):
                line = line.replace('\n','')
                identifiers.append(line)
            else:
                line = line.replace('\n','')
                sequences.append(line)
            
    return sequences

In [58]:
fasta_file = './Data/generated_inliers.fasta'
gi_ids = parse_fasta(fasta_file)
gi_seqs = parse_fasta2(fasta_file)

In [59]:
len(gi_idx)

507

In [60]:
gi_df = pd.DataFrame(gi_seqs, index=gi_ids)
gi_df.columns = ['Sequence']
gi_df.head(10)

,Sequence
gi_1,EKFKTLVRGSIIELARLRKCVKAGVGVALN
gi_2,PAVIIIFVPGFYIKTKNAARAICILRGLLSPCIATI
gi_3,LNFKKRKLLCGLGSCGLGKAPGLQQILSKWTVRSAAAG
gi_4,WFESLGKVAHGLCTTIKVCIAGGQRQGEPAAH
gi_5,FLPVPCSVKKLGGLKKHRRSWIHKLVNL
gi_6,VAKKLSLIGVVILVVLPKAGYFRATLLYHWC
gi_7,GGRLFSKGKCLKGAGALKAGVKSRTLVCVNSEK
gi_8,TGLIQSAGAFCFGKVGGRCGGKKE
gi_9,RPETIRKEKLIHTLSLAIVLGSTKPISKWNK
gi_10,SACSCKQVRSVAPMPNWDEKGKCYWAKKFGLLFK


In [61]:
gi_slct = gi_df.loc[gi_idx]
gi_slct.head(10)

,Sequence
gi_2,PAVIIIFVPGFYIKTKNAARAICILRGLLSPCIATI
gi_13,AQLGKGGAAPHLALAKIPKLFRLDPNAKCAIWAM
gi_32,DCAGFIKALDATLKQNEPKHAGIS
gi_37,ARACKQSCKMGKKRYLWTYKGISIKFPQKKKMLVFA
gi_42,PRRIVVCFNQLNIAPFVDTHWIAIRKVRSKSMTRAG
gi_46,RKITEHHISGKRYEKMTALPFAAKLTSVVKI
gi_52,VRLKDYGVPTGKRLKAINLLQPWK
gi_59,ICSLRLRGEPAGGGATFRQMESCKLAKLLISL
gi_66,CVLSCVAQPAYSLRGIYRRKRGCNACKTAKPFQVTCD
gi_87,LDFVRGLAKVKVQLILISCPME


In [62]:
gi_slct.to_csv('./Data/selected_inliers_sequences.csv')

In [63]:
def fasta_converter(database):
    '''Use the indices of the database and the column 'Sequence' to create a fasta file'''
    ofile = open('./Data/selected_inliers.fasta', "w")

    for i in range(len(database.Sequence)):

        ofile.write(">" + database.index[i] + "\n" +database.Sequence[i] + "\n")
        
    ofile.close()

In [64]:
fasta_converter(gi_slct)